# IIP Iceberg Sighting Database Information

In [1]:
url = lambda year: f'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/IIP_{year}IcebergSeason.csv'

In [2]:
url(2011)

'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G00807/IIP_2011IcebergSeason.csv'

In [11]:
from pandas import read_csv, to_datetime

In [4]:
df = read_csv(url(2011))

In [9]:
df.head(2)

,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE
0,2011,2.0,11/30/2010,1751.0,60.36,-62.88,RAD,RAD,RAD,GTJZ
1,2011,3.0,1/12/2011,0.0,60.50,-63.50,RAD,RAD,RAD,BOGU


In [19]:
def convert_str_column_to_datetime_column(df, *column_names):
    for column_name in column_names:
        df[column_name] = to_datetime(df[column_name])
    return df

In [195]:
def convert_float_column_to_int_column(df, *column_names, drop_nan_rows=True):
    for column_name in column_names:
        try:
            df[column_name] = df[column_name].astype(int)
        except ValueError:
            if drop_nan_rows:
                df = df.dropna(subset=[column_name]).reset_index(drop=True)
                df[column_name] = df[column_name].astype(int)
            else:
                raise
    return df

In [196]:
def convert_sighting_time_column_to_timedelta_column(df, time_column_name, timedelta_column_name):
    df[time_column_name] = df[time_column_name].astype(str).str.zfill(4)
    df[timedelta_column_name] = df[timedelta_column_name].str[0:2] + ':' + df[timedelta_column_name].str[2:4] + ':00'
    df[timedelta_column_name] = pd.to_timedelta(df[timedelta_column_name])
    return df

In [201]:
def create_sighting_datetime_column(df, date_column_name, timedelta_column_name, datetime_column_name):
    df[datetime_column_name] = df[date_column_name] + df[timedelta_column_name]
    return df

In [198]:
df2 = convert_float_column_to_int_column(df, 'ICEBERG_NUMBER', 'SIGHTING_TIME')

In [199]:
df3 = convert_sighting_time_column_to_timedelta_column(df2, 'SIGHTING_TIME', 'SIGHTING_TIMEDELTA')

In [202]:
df4 = create_sighting_datetime_column(df3, 'SIGHTING_DATE', 'SIGHTING_TIMEDELTA', 'SIGHTING_DATETIME')

In [203]:
df4.head(3)

,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,SIGHTING_TIMEDELTA,SIGHTING_DATETIME
0,2011,2,2010-11-30,1751,60.36,-62.88,RAD,RAD,RAD,GTJZ,17:51:00,2010-11-30 17:51:00
1,2011,3,2011-01-12,0,60.50,-63.50,RAD,RAD,RAD,BOGU,00:00:00,2011-01-12 00:00:00
2,2011,4,2011-07-30,723,49.84,-56.69,SAT-HIGH,LG,GEN,RSA1,07:23:00,2011-07-30 07:23:00
